In [1]:
import pandas as pd

In [2]:
from backtesting import Strategy
from backtesting.lib import crossover

/Users/samholt/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:44: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
from backtesting import Backtest

In [4]:
%matplotlib inline
import seaborn as sns
from Price_Data import hist_data

In [5]:
import matplotlib.pyplot as plt

In [6]:
btcusd = hist_data('Bitstamp')

/Users/samholt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [7]:
btcusd.columns = ['Open']

In [8]:
ss = btcusd.resample('H')

In [9]:
op = ss.first().ffill()
hi = ss.max().ffill()
lo = ss.min().ffill()
cl = ss.last().ffill()

In [10]:
btcusd

,Open
Date,
2011-09-13 13:53:36,5.80
2011-09-13 13:53:44,5.83
2011-09-13 13:53:49,5.90
2011-09-13 13:53:54,6.00
2011-09-13 14:32:53,5.95
...,...
2019-12-12 23:58:18,7184.44
2019-12-12 23:58:20,7184.44
2019-12-12 23:59:05,7190.53


In [11]:
ohlc = op
ohlc.columns = ['Open']
ohlc['High'] = hi
ohlc['Low'] = lo
ohlc['Close'] = cl
ohlc

,Open,High,Low,Close
Date,,,,
2011-09-13 13:00:00,5.80,6.00,5.80,6.00
2011-09-13 14:00:00,5.95,5.95,5.76,5.76
2011-09-13 15:00:00,5.65,5.65,5.65,5.65
2011-09-13 16:00:00,5.65,5.65,5.65,5.65
2011-09-13 17:00:00,5.65,5.65,5.65,5.65
...,...,...,...,...
2019-12-12 19:00:00,7205.66,7218.00,7186.36,7193.66
2019-12-12 20:00:00,7205.14,7250.23,7195.00,7242.49
2019-12-12 21:00:00,7245.85,7253.68,7180.83,7194.57


In [12]:
import datetime as dt

In [13]:
yr1 = ohlc.loc['2018-12-12':]
yr2 = ohlc.loc['2017-12-12':]
yr4 = ohlc.loc['2015-12-12':]

# 1 Year SMA and EMA

In [14]:
def SMA(values,n):
    return pd.Series(values).rolling(n).mean()

In [15]:
class SmaCross(Strategy):
    
    n1 = 37
    n2 = 73
    
    def init(self):
        # Precompute two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, buy the asset
        if crossover(self.sma1, self.sma2):
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif crossover(self.sma2, self.sma1):
            self.sell()

In [16]:
def EMA(val,n):
    res = pd.Series(val).copy()
    return res.ewm(span=n,adjust=False).mean()

In [17]:
class EmaCross(Strategy):
    
    n1 = 901
    n2 = 931
    
    def init(self):
        # Precompute two moving averages
        self.ema1 = self.I(EMA, self.data.Close, self.n1)
        self.ema2 = self.I(EMA, self.data.Close, self.n2)
        
    def next(self):
        # If sma1 crosses above ema2, buy the asset
        if crossover(self.ema1, self.ema2):
            self.buy()

        # Else, if ema1 crosses below ema2, sell it
        elif crossover(self.ema2, self.ema1):
            self.sell()

In [18]:
def DEMA(val,n):
    exp1 = EMA(val,n)
    exp2 = EMA(exp1,n)
    return 2*exp1 - exp2

In [19]:
class DemaCross(Strategy):
    
    n1 = 681
    n2 = 981
    
    def init(self):
        # Precompute two moving averages
        self.dema1 = self.I(DEMA, self.data.Close, self.n1)
        self.dema2 = self.I(DEMA, self.data.Close, self.n2)
        
    def next(self):
        # If sma1 crosses above ema2, buy the asset
        if crossover(self.dema1, self.dema2):
            self.buy()

        # Else, if ema1 crosses below ema2, sell it
        elif crossover(self.dema2, self.dema1):
            self.sell()

In [20]:
import scipy.interpolate as interp
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# Strategies for different Time Horizons

In [28]:
def time_horiz(hist,ma):
    
    if ma == 'SMA':
        bt = Backtest(hist,SmaCross,cash=10000)
    elif ma == 'EMA':
        bt = Backtest(hist,EmaCross,cash=10000)
    elif ma == 'DEMA':
        bt = Backtest(hist,DemaCross,cash=10000)
    
    data, heat = bt.optimize('Sharpe Ratio',n1=range(1,200,9),n2=range(1,200,9),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

    heat.sort_values()
    hmap = heat.groupby(['n1','n2']).mean().unstack()
    hmap

    heat_pic = sns.heatmap(hmap, cmap='Spectral')
    
    tr = hmap.stack()

    n1 = [i[0] for i in tr.index]
    n2 = [i[1] for i in tr.index]

    val = []
    for (i,j) in zip(n1,n2):
        if i<j:
            val.append(tr.loc[i,j])

    px, py = np.meshgrid(np.linspace(np.min(n1),np.max(n1),100),np.linspace(np.min(n2),np.max(n2),100))
    pz = interp.griddata((n1,n2),val,(px,py),method='cubic')

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    surf = ax.plot_surface(px,py,pz,cmap=cm.coolwarm)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.show()
    
    return data

# Example of walk forward for a year

In [41]:
Y_2018_to_2019 = ohlc.loc['2018-12-12':]
Y_2017_to_2018 = ohlc.loc['2017-12-12':'2018-12-12']
Backtest(Y_2017_to_2018,SmaCross,cash=10000)
stats_2017_to_2018, heat_2017_to_2018 = Backtest(Y_2017_to_2018,SmaCross,cash=10000).optimize('Sharpe Ratio',n1=range(1,20,10),n2=range(1,20,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)
print('Optimal parameters are ', stats_2017_to_2018._strategy)


stats_walk_forward_2018_to_2019 = Backtest(Y_2018_to_2019, SmaCross, cash=10000).run(n1=stats_2017_to_2018._strategy.n1, n2=stats_2017_to_2018._strategy.n2)
stats_walk_forward_2018_to_2019

Optimal parameters are  SmaCross(n1=1,n2=11)


Start                      2018-12-12 00:00:00
End                        2019-12-12 23:00:00
Duration                     365 days 23:00:00
Exposure [%]                           99.7381
Equity Final [$]                         17241
Equity Peak [$]                        20940.5
Return [%]                             72.4096
Buy & Hold Return [%]                  115.559
Max. Drawdown [%]                     -28.9398
Avg. Drawdown [%]                     -5.97389
Max. Drawdown Duration       103 days 06:00:00
Avg. Drawdown Duration         8 days 22:00:00
# Trades                                  1616
Win Rate [%]                           26.1139
Best Trade [%]                          25.772
Worst Trade [%]                       -6.40701
Avg. Trade [%]                       0.0445031
Max. Trade Duration            1 days 19:00:00
Avg. Trade Duration            0 days 06:00:00
Expectancy [%]                        0.834291
SQN                                   0.778608
Sharpe Ratio 

# 1 Year SMA

In [22]:
bt = Backtest(yr1,SmaCross,cash=10000)
data, heat = bt.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

/Users/samholt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Searching best of 4950 configurations.
  


In [23]:
Backtest(yr1,SmaCross,cash=10000).run()

Start                     2018-12-12 00:00:00
End                       2019-12-12 23:00:00
Duration                    365 days 23:00:00
Exposure [%]                          97.6318
Equity Final [$]                      27938.3
Equity Peak [$]                         28153
Return [%]                            179.383
Buy & Hold Return [%]                 115.559
Max. Drawdown [%]                    -36.0934
Avg. Drawdown [%]                    -4.24022
Max. Drawdown Duration      103 days 08:00:00
Avg. Drawdown Duration        6 days 02:00:00
# Trades                                  137
Win Rate [%]                          42.3358
Best Trade [%]                        26.2978
Worst Trade [%]                      -9.69474
Avg. Trade [%]                       0.888667
Max. Trade Duration           9 days 20:00:00
Avg. Trade Duration           2 days 15:00:00
Expectancy [%]                        3.46709
SQN                                   1.52872
Sharpe Ratio                      

In [24]:
data

Start                         2018-12-12 00:00:00
End                           2019-12-12 23:00:00
Duration                        365 days 23:00:00
Exposure [%]                              75.2135
Equity Final [$]                          30868.1
Equity Peak [$]                           38628.6
Return [%]                                208.681
Buy & Hold Return [%]                     115.559
Max. Drawdown [%]                        -40.2636
Avg. Drawdown [%]                        -2.66611
Max. Drawdown Duration          169 days 04:00:00
Avg. Drawdown Duration            3 days 18:00:00
# Trades                                        5
Win Rate [%]                                   40
Best Trade [%]                            205.001
Worst Trade [%]                          -10.2256
Avg. Trade [%]                            38.8915
Max. Trade Duration             154 days 13:00:00
Avg. Trade Duration              55 days 02:00:00
Expectancy [%]                            48.6797


In [25]:
heat.index.names = [r'Short (days)',r'Long (days)']

In [26]:
hh = heat*np.sqrt(12)

In [27]:
ax.get_xticks()

NameError: name 'ax' is not defined

In [ ]:
hh.sort_values()
hmap_sma = hh.groupby([r'Short (days)',r'Long (days)']).mean().unstack()
ax = sns.heatmap(hmap_sma,cmap='RdYlGn',xticklabels=list(range(0,101)),yticklabels=list(range(0,101)))
ax.set_xticks(ax.get_xticks()[::12])
ax.set_xticklabels(list(range(41))[::5])
ax.set_yticks(ax.get_yticks()[::12])
ax.set_yticklabels(list(range(0,41))[::5])
fig = ax.get_figure()

In [ ]:
fig.tight_layout()
fig.savefig('btc_sma1_hmap.png')

In [ ]:
tr = hmap_sma.stack()
n1 = [i[0] for i in tr.index]
n2 = [i[1] for i in tr.index]

val = []
for (i,j) in zip(n1,n2):
    if i<j:
        val.append(tr.loc[i,j])

px, py = np.meshgrid(np.linspace(np.min(n1),np.max(n1),300),np.linspace(np.min(n2),np.max(n2),300))
pz = interp.griddata((n1,n2),val,(px,py),method='cubic')

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(px,py,pz,cmap='RdYlGn',vmin=0.1,vmax=0.9)
fig.colorbar(surf, shrink=0.5,aspect=5)
ax.set_xlabel(r'Short (hrs)')
ax.set_ylabel(r'Long (hrs)')
ax.set_zlabel('Sharpe ratio')
ax.set_zticks([-0.5,0,0.5,1,1.5])
ax.set_xticks([0,250,500,750,1000])
ax.set_yticks([0,250,500,750,1000])
plt.tight_layout()
plt.savefig('plt_sma.png')

In [ ]:
rr = Backtest(yr1,SmaCross_opt,cash=10000).run()._trade_data
rr['P/L'].iloc[0] = 0
rr['P/L'].interpolate(method='time',inplace=True)

In [ ]:
plt.xlabel('Date')
plt.ylabel(r'Equity (USD)')
fig = rr['Equity'].plot().get_figure()
fig.tight_layout()
fig.savefig('btc_sma1_equity.png')

# Price Data Plots

In [ ]:
import yfinance as yf

In [ ]:
sp_df = yf.download('^GSPC', 
                      start='2011-09-13', 
                      end='2019-12-13', 
                      progress=False,interval='1d')
sp = sp_df.iloc[:,:4]
sp

In [ ]:
sp1 = sp.loc['2018-12-12':]
sp2 = sp.loc['2017-12-12':]
sp4 = sp.loc['2015-12-12':]

In [ ]:
class SmaCross_alt(Strategy):
    
    n1 = 40
    n2 = 43
    
    def init(self):
        # Precompute two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, buy the asset
        if crossover(self.sma1, self.sma2):
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif crossover(self.sma2, self.sma1):
            self.sell()

In [ ]:
sp_run = Backtest(sp1,SmaCross_alt,cash=10000).run()
val = sp_run._trade_data

In [ ]:
plt.plot(sp['Close'])
plt.xlabel('Date')
plt.ylabel('Spot Price (USD)')
plt.tight_layout
plt.savefig('sp_price_plt.png')

In [ ]:
plt.plot(ohlc['Close'])
plt.xlabel('Date')
plt.ylabel('Spot Price (USD)')
plt.tight_layout
plt.savefig('btc_price_plt.png')

In [ ]:
plt.plot(yr1['Close'])
plt.xlabel('Date')
plt.ylabel('BTCUSD Spot Price (USD)')
plt.tight_layout
plt.savefig('btc_yr1_p.png')

In [ ]:
rr['Equity'].plot(label='BTCUSD')
val['Equity'].plot(label=r'S&P 500')
plt.ylabel(r'Equity (USD)')
plt.legend(loc='upper left')
plt.tight_layout()
plt.savefig('equity.png')

# Traditional Windows

In [ ]:
Backtest(yr1,SmaCross,cash=10000).optimize('Sharpe Ratio', n1=240,n2=960)

In [ ]:
Backtest(yr1,EmaCross,cash=10000).optimize('Sharpe Ratio', n1=240,n2=960)

In [ ]:
Backtest(yr1,DemaCross,cash=10000).optimize('Sharpe Ratio', n1=240,n2=960)

In [ ]:
Backtest(sp1,SmaCross,cash=10000).optimize('Sharpe Ratio', n1=10,n2=40)

In [ ]:
Backtest(sp1,EmaCross,cash=10000).optimize('Sharpe Ratio', n1=10,n2=40)

In [ ]:
0.451627*np.sqrt(12)

In [ ]:
Backtest(sp1,DemaCross,cash=10000).optimize('Sharpe Ratio', n1=10,n2=40)

# PnL Plots

In [ ]:
class SmaCross_trad(Strategy):
    
    n1 = 1
    n2 = 21
    
    def init(self):
        # Precompute two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, buy the asset
        if crossover(self.sma1, self.sma2):
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif crossover(self.sma2, self.sma1):
            self.sell()

In [ ]:
alt_max = Backtest(yr1,SmaCross_alt,cash=10000).run()._trade_data
alt_max['P/L'].iloc[0] = 0
alt_max = alt_max['P/L'].interpolate(method='time')

In [ ]:
plt.xlabel('Date')
plt.ylabel(r'PnL (USD)')
rr = Backtest(yr1,SmaCross_opt,cash=10000).run()._trade_data
rr['P/L'].iloc[0] = 0
rr = rr['P/L'].interpolate(method='time')
fig = rr.plot().get_figure()
fig.tight_layout()
fig.savefig('btc_sma1_pl.png')

# Correlation Analysis

In [ ]:
rr = Backtest(yr1,SmaCross_opt,cash=10000).run()._trade_data
ap = rr['P/L']
bp = val['P/L']
bp[0] = 0
ap[0] = 0

In [ ]:
bp.interpolate(method='time',inplace=True)
ap.interpolate(method='time',inplace=True)

In [ ]:
ap = ap.asfreq('D')

In [ ]:
bp = bp.asfreq('D').interpolate(method='time')
bp.rolling(20).corr(ap).dropna().plot()
plt.ylabel('Correlation')
plt.tight_layout()

In [ ]:
bp.rolling(20).corr(ap).dropna().plot()
plt.ylabel('Correlation')
plt.tight_layout()
plt.savefig('cor_sma1.png')

In [ ]:
sp1_close = sp1['Close']
btc1_close = yr1['Close']

In [ ]:
btc1_close = btc1_close.asfreq('D')
sp1_close = sp1_close.asfreq('D')
sp1_close.interpolate(method='time',inplace=True)

In [ ]:
import matplotlib.dates as mdates

In [ ]:
btc1_close.rolling(20).corr(sp1_close).dropna().plot()
plt.ylabel('Correlation')
plt.axvspan(dt.datetime(2019,5,8),dt.datetime(2019,6,20), color='r', alpha=0.3)
plt.axvspan(dt.datetime(2019,1,1),dt.datetime(2019,2,12), color='r', alpha=0.3)
plt.axvspan(dt.datetime(2019,7,8),dt.datetime(2019,8,20), color='r', alpha=0.3)
plt.axvspan(dt.datetime(2019,11,12),dt.datetime(2019,12,12), color='r', alpha=0.3)
plt.tight_layout()
plt.savefig('cor_price.png')

In [ ]:
ap.rolling(20).corr(sp1_close).dropna().plot()
plt.ylabel('Correlation')
plt.axvspan(dt.datetime(2019,5,8),dt.datetime(2019,6,10), color='r', alpha=0.3)
plt.axvspan(dt.datetime(2019,9,24),dt.datetime(2019,10,16), color='r', alpha=0.3)
plt.axvspan(dt.datetime(2019,11,4),dt.datetime(2019,12,12), color='r', alpha=0.3)
plt.tight_layout()
plt.savefig('sp_close_sma1_corr.png')

# Evolving Sharpe Ratio

In [ ]:
y1 = ohlc.loc['2018-12-12':]
y2 = ohlc.loc['2017-12-12':'2018-12-12']
y3 = ohlc.loc['2016-12-12':'2017-12-12']
y4 = ohlc.loc['2015-12-12':'2016-12-12']
y5 = ohlc.loc['2014-12-12':'2015-12-12']
y6 = ohlc.loc['2013-12-12':'2014-12-12']
y7 = ohlc.loc['2012-12-12':'2013-12-12']
y8 = ohlc.loc['2011-12-12':'2012-12-12']

## SMA

In [ ]:
sma = Backtest(y1,SmaCross,cash=10000)

In [ ]:
s1 = sma.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
sma = Backtest(y2,SmaCross,cash=10000)

In [ ]:
s2 = sma.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
sma = Backtest(y3,SmaCross,cash=10000)

In [ ]:
s3 = sma.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
sma = Backtest(y4,SmaCross,cash=10000)

In [ ]:
s4 = sma.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
sma = Backtest(y5,SmaCross,cash=10000)

In [ ]:
s5 = sma.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
sma = Backtest(y6,SmaCross,cash=10000)
s6 = sma.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
sma = Backtest(y7,SmaCross,cash=10000)
s7 = sma.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
sma = Backtest(y8,SmaCross,cash=10000)
s8 = sma.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
ema = Backtest(y2,EmaCross,cash=10000)
e2 = ema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
ema = Backtest(y3,EmaCross,cash=10000)
e3 = ema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
ema = Backtest(y4,EmaCross,cash=10000)
e4 = ema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
ema = Backtest(y5,EmaCross,cash=10000)
e5 = ema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
ema = Backtest(y6,EmaCross,cash=10000)
e6 = ema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
ema = Backtest(y7,EmaCross,cash=10000)
e7 = ema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
ema = Backtest(y8,EmaCross,cash=10000)
e8 = ema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
dema = Backtest(y2,DemaCross,cash=10000)
d2 = dema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
dema = Backtest(y3,DemaCross,cash=10000)
d3 = dema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
dema = Backtest(y4,DemaCross,cash=10000)
d4 = dema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
dema = Backtest(y5,DemaCross,cash=10000)
d5 = dema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
dema = Backtest(y6,DemaCross,cash=10000)
d6 = dema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
dema = Backtest(y7,DemaCross,cash=10000)
d7 = dema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
dema = Backtest(y8,DemaCross,cash=10000)
d8 = dema.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2)

In [ ]:
d8

In [ ]:
from dateutil.relativedelta import relativedelta as rd

In [ ]:
start = s1['Start']
times_arr = [start]

for i in range(1,8):
    times_arr.append(start-rd(years=i))
    
times_arr

In [ ]:
sma_s = [201,441,251,311,151,361,831,211]
sma_l = [951,521,481,991,861,991,991,541]

In [ ]:
ema_s = [901,101,821,881,421,541,161,521]
ema_l = [931,211,951,961,791,831,931,681]

In [ ]:
dema_s = [681,131,421,921,401,331,531,981]
dema_l = [981,701,461,951,571,341,661,991]

In [ ]:
for i in [sma_s,sma_l,ema_s,ema_l,dema_s,dema_l]:
    print(np.mean(i))

In [ ]:
q1 = pd.Series(sma_s)
q2 = pd.Series(sma_l)
q1.corr(q2)

In [ ]:
plt.plot(times_arr,sma_s,label='SMA Short')
plt.plot(times_arr,sma_l, label = 'SMA Long')
plt.xlabel('Date')
plt.ylabel(r'Rolling Window (hrs)')
plt.legend(loc=(0.07,0.04))
plt.tight_layout()
plt.savefig('sma_rol.png')

In [ ]:
plt.plot(times_arr,ema_s,label='EMA Short')
plt.plot(times_arr,ema_l,label='EMA Long')
plt.xlabel('Date')
plt.ylabel(r'Rolling Window (hrs)')
plt.legend(loc=(0.1,0.6))
plt.tight_layout()
plt.savefig('ema_rol.png')

In [ ]:
plt.plot(times_arr,dema_s,label='DEMA Short')
plt.plot(times_arr,dema_l, label='DEMA Long')
plt.xlabel('Date')
plt.ylabel(r'Rolling Window (hrs)')
plt.legend()
plt.tight_layout()
plt.savefig('dema_rol.png')

# Cumulative PnL

In [ ]:
class SmaCross_opt(Strategy):
    
    n1 = 201
    n2 = 951
    
    def init(self):
        # Precompute two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, buy the asset
        if crossover(self.sma1, self.sma2):
            self.buy()

        # Else, if sma1 crosses below sma2, sell it
        elif crossover(self.sma2, self.sma1):
            self.sell()

In [ ]:
class EmaCross_opt(Strategy):
    
    n1 = 901
    n2 = 931
    
    def init(self):
        # Precompute two moving averages
        self.ema1 = self.I(EMA, self.data.Close, self.n1)
        self.ema2 = self.I(EMA, self.data.Close, self.n2)
        
    def next(self):
        # If sma1 crosses above ema2, buy the asset
        if crossover(self.ema1, self.ema2):
            self.buy()

        # Else, if ema1 crosses below ema2, sell it
        elif crossover(self.ema2, self.ema1):
            self.sell()

In [ ]:
class DemaCross_opt(Strategy):
    
    n1 = 681
    n2 = 981
    
    def init(self):
        # Precompute two moving averages
        self.dema1 = self.I(DEMA, self.data.Close, self.n1)
        self.dema2 = self.I(DEMA, self.data.Close, self.n2)
        
    def next(self):
        # If sma1 crosses above ema2, buy the asset
        if crossover(self.dema1, self.dema2):
            self.buy()

        # Else, if ema1 crosses below ema2, sell it
        elif crossover(self.dema2, self.dema1):
            self.sell()

In [ ]:
spl = Backtest(yr1,SmaCross_opt,cash=10000).run()._trade_data

In [ ]:
spl = spl['P/L']

In [ ]:
spl.fillna(0).cumsum().plot()

In [ ]:
epl = Backtest(yr1,EmaCross_opt,cash=10000).run()._trade_data
dpl = Backtest(yr1,DemaCross_opt,cash=10000).run()._trade_data

In [ ]:
epl = epl['P/L']
dpl = dpl['P/L']

In [ ]:
spl.fillna(0).cumsum().plot(label = 'SMA Strategy')
epl.fillna(0).cumsum().plot(label = 'EMA Strategy')
dpl.fillna(0).cumsum().plot(label = 'DEMA Strategy')
plt.ylabel(r'PnL (USD)')
plt.legend(loc = 'upper left')
plt.tight_layout()
plt.savefig('cum_pnl.png')

# 1 Year EMA

In [ ]:
ema1 = Backtest(yr1,EmaCross,cash=10000)
d_ema1, heat_ema = ema1.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
d_ema1

In [ ]:
heat_ema.index.names = [r'Short (days)',r'Long (days)']
hh_ema = heat_ema*np.sqrt(12)

In [ ]:
hh_ema.sort_values()
hmap_ema = hh_ema.groupby([r'Short (days)',r'Long (days)']).mean().unstack()
ax = sns.heatmap(hmap_ema,cmap='RdYlGn',xticklabels=list(range(0,101)),yticklabels=list(range(0,101)))
ax.set_xticks(ax.get_xticks()[::12])
ax.set_xticklabels(list(range(0,41))[::5])
ax.set_yticks(ax.get_yticks()[::12])
ax.set_yticklabels(list(range(0,41))[::5])
fig = ax.get_figure()

In [ ]:
fig.tight_layout()
fig.savefig('btc_ema_hmap.png')

In [ ]:
tr = hmap_ema.stack()
n1 = [i[0] for i in tr.index]
n2 = [i[1] for i in tr.index]

val = []
for (i,j) in zip(n1,n2):
    if i<j:
        val.append(tr.loc[i,j])

px, py = np.meshgrid(np.linspace(np.min(n1),np.max(n1),300),np.linspace(np.min(n2),np.max(n2),300))
pz = interp.griddata((n1,n2),val,(px,py),method='cubic')

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(px,py,pz,cmap='RdYlGn',vmin=0.1,vmax=0.9)
fig.colorbar(surf, shrink=0.5,aspect=5)
ax.set_xlabel(r'Short (hrs)')
ax.set_ylabel(r'Long (hrs)')
ax.set_zlabel('Sharpe ratio')
ax.set_zticks([-0.5,0,0.5,1,1.5])
ax.set_xticks([0,250,500,750,1000])
ax.set_yticks([0,250,500,750,1000])
plt.tight_layout()
plt.savefig('plt_ema.png')

# 1 Year DEMA

In [ ]:
dema1 = Backtest(yr1,DemaCross,cash=10000)
d_dema1, heat_dema = dema1.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
d_dema1

In [ ]:
heat_dema.index.names = [r'Short (days)',r'Long (days)']
hh_dema = heat_dema*np.sqrt(12)

In [ ]:
hh_dema.sort_values()
hmap_dema = hh_dema.groupby([r'Short (days)',r'Long (days)']).mean().unstack()
ax = sns.heatmap(hmap_dema,cmap='RdYlGn',xticklabels=list(range(0,101)),yticklabels=list(range(0,101)))
ax.set_xticks(ax.get_xticks()[::12])
ax.set_xticklabels(list(range(0,41))[::5])
ax.set_yticks(ax.get_yticks()[::12])
ax.set_yticklabels(list(range(0,41))[::5])
fig = ax.get_figure()

In [ ]:
fig.tight_layout()
fig.savefig('btc_dema_hmap.png')

In [ ]:
tr = hmap_dema.stack()
n1 = [i[0] for i in tr.index]
n2 = [i[1] for i in tr.index]

val = []
for (i,j) in zip(n1,n2):
    if i<j:
        val.append(tr.loc[i,j])

px, py = np.meshgrid(np.linspace(np.min(n1),np.max(n1),300),np.linspace(np.min(n2),np.max(n2),300))
pz = interp.griddata((n1,n2),val,(px,py),method='cubic')

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(px,py,pz,cmap='RdYlGn',vmin=-0.3,vmax=0.5)
fig.colorbar(surf, shrink=0.5,aspect=5)
ax.set_xlabel(r'Short (hrs)')
ax.set_ylabel(r'Long (hrs)')
ax.set_zlabel('Sharpe ratio')
ax.set_zticks([-1,-0.5,0,0.5,1])
ax.set_xticks([0,250,500,750,1000])
ax.set_yticks([0,250,500,750,1000])
plt.tight_layout()
plt.savefig('plt_dema.png')

# 2 Year SMA

In [ ]:
sma2 = Backtest(yr2,SmaCross,cash=10000)
d_sma2, heat_sma2 = sma2.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
d_sma2

In [ ]:
heat_sma2.sort_values()
hmap_sma2 = heat_sma2.groupby(['n1','n2']).mean().unstack()
sns.heatmap(hmap_sma2,cmap='RdYlGn')

# 2 Year EMA

In [ ]:
ema2 = Backtest(yr2,EmaCross,cash=10000)
d_ema2, heat_ema2 = ema2.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
d_ema2

In [ ]:
heat_ema2.sort_values()
hmap_ema2 = heat_ema2.groupby(['n1','n2']).mean().unstack()
sns.heatmap(hmap_ema2,cmap='RdYlGn')

# 2 Year DEMA

In [ ]:
dema2 = Backtest(yr2,DemaCross,cash=10000)
d_dema2, heat_dema2 = dema2.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
d_dema2

In [ ]:
heat_dema2.sort_values()
hmap_dema2 = heat_dema2.groupby(['n1','n2']).mean().unstack()
sns.heatmap(hmap_dema2,cmap='RdYlGn')

# 4 Year SMA

In [ ]:
sma4 = Backtest(yr4,SmaCross,cash=10000)
d_sma4, heat_sma4 = sma4.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
d_sma4

In [ ]:
heat_sma4.sort_values()
hmap_sma4 = heat_sma4.groupby(['n1','n2']).mean().unstack()
sns.heatmap(hmap_sma4,cmap='RdYlGn')

# 4 Year EMA

In [ ]:
ema4 = Backtest(yr4,EmaCross,cash=10000)
d_ema4, heat_ema4 = ema4.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
d_ema4

# 4 Year DEMA

In [ ]:
dema4 = Backtest(yr4,DemaCross,cash=10000)
d_dema4, heat_dema4 = dema4.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
d_dema4

# Whole Dataset

In [ ]:
smaw = Backtest(ohlc,SmaCross,cash=10000)
d_smaw, heat_smaw = smaw.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
emaw = Backtest(ohlc,EmaCross,cash=10000)
d_emaw, heat_emaw = emaw.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
demaw = Backtest(ohlc,DemaCross,cash=10000)
d_demaw, heat_demaw = demaw.optimize('Sharpe Ratio',n1=range(1,1000,10),n2=range(1,1000,10),constraint=lambda p: p.n1<p.n2,return_heatmap=True)

In [ ]:
d_demaw